# BlazingSQL + cuML NYC Taxi Cab Fare Prediction

This demo uses pubically availible [NYC Taxi Cab Data](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction) to predict the total fare of a taxi ride in New York City given the pickup and dropoff locations. 

In this notebook, we will cover: 
- How to read and query csv files with cuDF and BlazingSQL.
- How to implement a linear regression model with cuML.

## Imports

In [ ]:
import os
from cuml import LinearRegression
from blazingsql import BlazingContext

## Create BlazingContext
You can think of the BlazingContext much like a SparkContext, this is where information such as FileSystems you have registered and Tables you have created will be stored. 

In [2]:
# start up BlazingSQL 
bc = BlazingContext()

lo
BlazingContext ready


### Download Data
For this demo we will train our model with 25,000,000 rows of data from 5 csv files (5,000,000 rows each). 

The cell below will download them from AWS for you.

In [2]:
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_00.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_01.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_02.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_03.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_04.csv    

## Extract, transform, load
To train our Linear Regression model, we must first ETL our data into GPU memory. BlazingContext achieves this simply with .[create_table()](https://docs.blazingdb.com/docs/using-blazingsql#section-create-tables), which locates data via the full file path. The cell below identifies the path to this notebook, then adds a wildcard `*` which allows BlazingContext to read all 5 CSVs at once.

In [ ]:
# find current working directory 
cwd = os.getcwd()
# add / & taxi wildcard to this directory
data_path = cwd + '/taxi*'
# what's the final path?
data_path

### ETL: Read and Join CSVs

In [ ]:
%%time
# tag column names & types
col_names = ['key', 'fare_amount', 'pickup_longitude', 'pickup_latitude', 
             'dropoff_longitude', 'dropoff_latitude', 'passenger_count']
col_types = ['date64', 'float32', 'float32', 'float32',
             'float32', 'float32', 'float32']

# create a table from all 'taxi*' files
bc.create_table('taxi', data_path, names=col_names, dtype=col_types)

In [4]:
# query the first 100 rows & display last 5
bc.sql("select * from taxi limit 100").tail()  # note: BlazingSQL queries return cuDF DataFrame results

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-02-02 22:30:19.002,8.9,-73.988708,40.758804,-73.986519,40.737202,1.0
1,2014-09-20 07:19:24.001,4.0,-73.990204,40.746708,-73.994728,40.750515,1.0
2,2013-02-23 07:18:05.001,5.5,-74.016762,40.709438,-74.009003,40.719498,3.0
3,2015-04-18 23:49:27.009,13.5,-74.002708,40.733730,-73.986099,40.734776,1.0
4,2010-03-04 08:15:59.001,10.5,-73.988365,40.737663,-74.012459,40.713932,1.0


### ETL: Query Table for Training Data
BlazingSQL allows but does not require capitalized SQL statements, mismatched combinations also work.  
A few examples:
- SELECT colA FROM table WHERE condition
- select colA from table where condition
- select colA from table WHERE condition

In [2]:
%%time
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(dayofmonth(key) as float) days,
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            taxi
        '''

# run query on table (returns cuDF DataFrame)
X_train = bc.sql(query)

# fill null values 
print(f'TODO\nis necessary?')
# X_train['longitude_distance'] = X_train['longitude_distance'].fillna(0)
print(len(X_train.loc[X_train['longitude_distance'].isna()==True]))
# X_train['latitude_distance'] = X_train['latitude_distance'].fillna(0)
print(len(X_train.loc[X_train['latitude_distance'].isna()==True]))
# X_train['passenger_count'] = X_train['passenger_count'].fillna(0)
print(len(X_train.loc[X_train['passenger_count'].isna()==True]))

In [ ]:
# how's it look? 
X_train.head()

In [ ]:
%%time
# query dependent variable y
y_train = bc.sql('SELECT fare_amount FROM main.taxi')

In [14]:
# how's it look?
y_train.head()

20196


,fare_amount
0,8.9
1,4.0
2,5.5
3,13.5
4,10.5


## Linear Regression
### LR: Train Model

In [15]:
%%time

import cuml
from cuml import LinearRegression

#create model
# lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = "eig")
lr = LinearRegression()

# train model on the first 1,700,000 rows (most my memory can fit)
reg = lr.fit(X_train[:17000000], y_train[:17000000])

# display results
print(f"Coefficients:\n{reg.coef_}\n")
print(f"Y intercept:\n{reg.intercept_}\n")

ModuleNotFoundError: No module named 'cuml'

### LR: Use Model to Predict Future Taxi Fares 

For this we are using a second dataset with no fare amount. The cell below will download this dataset for you.

In [18]:
# download test data
!wget 'https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv'

--2019-11-25 20:21:20--  https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv
Resolving blazingsql-demos.s3-us-west-1.amazonaws.com (blazingsql-demos.s3-us-west-1.amazonaws.com)... 52.219.120.129
Connecting to blazingsql-demos.s3-us-west-1.amazonaws.com (blazingsql-demos.s3-us-west-1.amazonaws.com)|52.219.120.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982916 (960K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 959.88K  4.48MB/s    in 0.2s    

2019-11-25 20:21:21 (4.48 MB/s) - ‘test.csv’ saved [982916/982916]



In [19]:
# create test table test CSV
bc.create_table('test', cwd+'/test.csv', names=col_names, dtype=col_types)

In [20]:
# extract time columns, long & lat, # riders (all floats)
query = '''
        select 
            cast(hour(key) as float) hours, 
            cast(month(key) as float) months, 
            cast(year(key) - 2000 as float) years,  
            cast(dropoff_longitude - pickup_longitude as float) longitude_distance, 
            cast(dropoff_latitude - pickup_latitude as float) latitude_distance, 
            cast(passenger_count as float) passenger_count
        from 
            test
        '''

# run query on table (returns cuDF DataFrame)
X_test = bc.sql(query)

# fill null values 
X_test['longitude_distance'] = X_test['longitude_distance'].fillna(0)
X_test['latitude_distance'] = X_test['latitude_distance'].fillna(0)
X_test['passenger_count'] = X_test['passenger_count'].fillna(0)

# how's it look? 
X_test.head()

20196


,hours,months,years,longitude_distance,latitude_distance,passenger_count
0,13.0,1.0,15.0,-0.008110,-0.019970,1.0
1,13.0,1.0,15.0,-0.012024,0.019814,1.0
2,11.0,10.0,11.0,0.002869,-0.005119,1.0
3,21.0,12.0,12.0,-0.009277,-0.016178,1.0
4,21.0,12.0,12.0,-0.022537,-0.045345,1.0


### Make Predictions 
- check csv for actual values
- are we going to compare or just predict?

In [21]:
# predict fares 
predictions = lr.predict(X_test_gdf)

# display 7 predictions
predictions.to_pandas().sample(5)

NameError: name 'lr' is not defined

In [19]:
# combine into a table of table points and predictions
X_test['predicted_fare'] = predictions

# how's that look? (pd sample)
X_test.to_pandas().sample(5)

,hours,months,years,longitude_distance,latitude_distance,passenger_count,predicted_fare
0,13.0,1.0,15.0,-0.008110,-0.019970,1.0,12.778599
1,13.0,1.0,15.0,-0.012024,0.019814,1.0,12.778547
2,11.0,10.0,11.0,0.002869,-0.005119,1.0,11.284673
3,21.0,12.0,12.0,-0.009277,-0.016178,1.0,11.864964
4,21.0,12.0,12.0,-0.022537,-0.045345,1.0,11.864986


## Real Life Example  
Predict cost of a ride from Grand Central Station to Samsung Next NYC at 7:00 AM on May 15th, 2020.
- needs adjusting in instance

In [20]:
# build a dataframe with cuDF
samsung_ride = cudf.DataFrame([('hours', 7.0), ('months', 5.0), 
                               ('years', 20.0), ('longitude_distance', 0.012727), 
                               ('latitude_distance', 0.008484), ('passenger_count', 1.0)])

samsung_ride['hours'] = samsung_ride['hours'].astype('float32')

samsung_ride['months'] = samsung_ride['months'].astype('float32')
samsung_ride['years'] = samsung_ride['years'].astype('float32')
samsung_ride['longitude_distance'] = samsung_ride['longitude_distance'].astype('float32')
samsung_ride['latitude_distance'] = samsung_ride['latitude_distance'].astype('float32')
samsung_ride['passenger_count'] = samsung_ride['passenger_count'].astype('float32')

# make prediction
samsung_prediction = lr.predict(samsung_ride)

# output fare prediction
samsung_prediction

0    16.517778
dtype: float32